In [1]:
import torch
import torchvision
from torchvision.transforms import v2
from torchvision.models import efficientnet_b0,EfficientNet_B0_Weights,densenet121,DenseNet121_Weights
from torch.utils.data import DataLoader
import skorch
from skorch.helper import predefined_split
from skorch.callbacks import Checkpoint,Freezer

c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feed

In [2]:
#Data first preprocessed and then split into train/val/test with ratios 0.8/0.1/0.1
#Data preprocessing is augmentation and normalization. The data will be resized according to model input size then
#Rotation, shearing and zooming were used to increase dataset size from 372 to 4000 images. Then normalization was applied and
#Dataset was split using ratio 0.8/0.1/0.1 for train/valid/test set. Split-folder python library was used.
#First 10 layers of models were frozen, last 10 layers will be fine-tuned and final classifier layer changed to linear layer

#Models trained individually and then added to KONet, efficientnetbo dropout ratio of 0.1 and densenet121 dropout ratio of 0.3

#First 10 layers will be frozen and the rest will be fine tuned but paper says only last 10 layers will be fine tuned.
#Unknown what is considered as a layer and what isn't.
#We will instead freeze roughly the first half of the layers in the model.
#We will freeze first Conv2dNormActivation and first nine MBConv blocks

In [14]:
n_classes=2
image_shape=224
augmented_dataset_size=4000
model_name='dense'
path="D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray Dataset"
non_augment_transform=v2.Compose([v2.ToImageTensor(),
                       v2.ToDtype(torch.float32),
                       v2.Resize((image_shape,image_shape),antialias=True),
                       v2.Normalize(mean=[0],std=[1]),
                       ])
transforms=v2.Compose([v2.ToImageTensor(),
                       v2.ToDtype(torch.float32),
                       v2.RandomAffine(degrees=30,shear=30),
                       v2.RandomZoomOut(side_range=(1,1.5)),
                       v2.Resize((image_shape,image_shape),antialias=True),
                       v2.Normalize(mean=[0],std=[1]),
                       ])

In [15]:
non_augmented_dataset=torchvision.datasets.ImageFolder(path,transform=non_augment_transform)
dataset=torchvision.datasets.ImageFolder(path,transform=transforms)

In [17]:
factor=augmented_dataset_size//len(dataset)-1
remainder=augmented_dataset_size-(factor+1)*len(dataset)

In [19]:
new_dataset=torch.utils.data.ConcatDataset([non_augmented_dataset]+[dataset for _ in range(factor)])
del non_augmented_dataset,dataset

In [4]:
#dataset=torchvision.datasets.ImageFolder(path,transform=transforms)
generator1 = torch.Generator().manual_seed(42)
train_set,valid_set,test_set=torch.utils.data.random_split(new_dataset, [0.8,0.1,0.1], generator=generator1)

In [5]:
image,label=train_set.__getitem__(0)

In [6]:
image.shape

torch.Size([3, 224, 224])

In [7]:
#EfficientNetB0 has 16 MBConv layers, freeze till 8th MBConv layer then. Freeze all till before 5th sequential
#DenseNet121 has 58 dense layers, freeze till 29th dense layer then. #Till before dense block 3
if model_name=='efficient':
    model=efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
    p=0.1
    model.classifier[0]=torch.nn.Dropout(p=p,inplace=True)
    model.classifier[-1]=torch.nn.Linear(in_features=1280,out_features=n_classes)
    freeze=['features.{}*.weight'.format(i) for i in range(5)] + ['features.{}*.bias'.format(i) for i in range(5)]
elif model_name=='dense':
    model=densenet121(weights=DenseNet121_Weights.DEFAULT)
    p=0.3
    model.classifier=torch.nn.Sequential(torch.nn.Dropout(p=p,inplace=True),
                                         torch.nn.Linear(in_features=1024,out_features=n_classes),
                                        )
    freeze=['features.conv0.weight','features.conv0.bias','features.norm0.weight','features.norm0.bias',
            'features.denseblock1.*.weight','features.denseblock1.*.bias','features.denseblock2.*.weight','features.denseblock2.*.bias',
            ]
    freeze+=['features.denseblock3.denselayer{}.*.weight'.format(i) for i in range(1,12)]
    freeze+=['features.denseblock3.denselayer{}.*.bias'.format(i) for i in range(1,12)]

In [8]:
model.features

Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(96, 128, ke

In [9]:
freeze

['features.conv0.weight',
 'features.conv0.bias',
 'features.norm0.weight',
 'features.norm0.bias',
 'features.denseblock1.*.weight',
 'features.denseblock1.*.bias',
 'features.denseblock2.*.weight',
 'features.denseblock2.*.bias',
 'features.denseblock3.denselayer1.*.weight',
 'features.denseblock3.denselayer2.*.weight',
 'features.denseblock3.denselayer3.*.weight',
 'features.denseblock3.denselayer4.*.weight',
 'features.denseblock3.denselayer5.*.weight',
 'features.denseblock3.denselayer6.*.weight',
 'features.denseblock3.denselayer7.*.weight',
 'features.denseblock3.denselayer8.*.weight',
 'features.denseblock3.denselayer9.*.weight',
 'features.denseblock3.denselayer10.*.weight',
 'features.denseblock3.denselayer11.*.weight',
 'features.denseblock3.denselayer1.*.bias',
 'features.denseblock3.denselayer2.*.bias',
 'features.denseblock3.denselayer3.*.bias',
 'features.denseblock3.denselayer4.*.bias',
 'features.denseblock3.denselayer5.*.bias',
 'features.denseblock3.denselayer6.*.bia

In [10]:
monitor = lambda net: any(net.history[-1, ('valid_accuracy_best','valid_loss_best')])
cp=Checkpoint(monitor='valid_loss_best',dirname='model',f_params=f'{model_name}best_param.pkl',
               f_optimizer=f'{model_name}best_opt.pkl', f_history=f'{model_name}best_history.json')
cb = skorch.callbacks.Freezer(freeze)
classifier = skorch.NeuralNetClassifier(
        model,
        criterion=torch.nn.CrossEntropyLoss(),
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(valid_set),
        iterator_train=DataLoader,
        iterator_valid=DataLoader,
        iterator_train__shuffle=True,
        iterator_train__pin_memory=True,
        iterator_valid__pin_memory=True,
        #iterator_train__num_workers=1,
        #iterator_valid__num_workers=1,
        #iterator_train__persistent_workers=True,
        #iterator_valid__persistent_workers=True,
        batch_size=32,
        device='cuda',
        callbacks=[cp,cb],#Try to implement accuracy and f1 score callables here
        warm_start=True,
        )

In [28]:
classifier.fit(train_set,y=None,epochs=2)

      2        2.1018       0.4054  23998925.0270        24.0298
      3        0.9049       0.5946  5892183.6486        24.2616


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
        

In [179]:
test=torch.ones((3,3,image_shape,image_shape))
out=classifier.predict(test)
print("Output shape: ",out.shape)

Output shape:  (3,)


In [ ]:
num_layers=2
linear_idcs=list(range(0, (num_layers+1)*3, 3))